# Yandex.Afisha - marketing expenses

Project Description-  
    
In this Yandex.Afisha. project I will use various analysis to help optimize marketing expenses. 

I will be using three data tables:
1. Server logs with data on Yandex.Afisha visits from June 2017 through May 2018
2. Dump file with all orders for the period
3. Marketing expenses statistics

To study: 
- How people use the product
- When they start to buy
- How much money each customer brings
- When they pay off

Using metrics like DAU, WAU, MAU, LTV, CAC, ROI, and more.


In [ ]:
!pip install sidetable

In [ ]:
import pandas as pd
import sidetable as stb
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## . Download the data and prepare it for analysis  
**Store the data on visits, orders, and expenses in variables. Optimize the data for analysis. Make sure each column contains the correct data type.**

In [ ]:
# downloading and getting to know first table
visits = pd.read_csv('/datasets/visits_log_us.csv')
visits.head()

In [ ]:
# renaming columns
visits.columns = visits.columns.str.lower().str.replace(' ', '_')
visits.head()

In [ ]:
visits.info(memory_usage='deep')

**Optimizing data by getting rid of the object types**

In [ ]:
visits['device'].value_counts()

In [ ]:
visits['device'] = visits['device'].astype('category')
visits['start_ts'] =  pd.to_datetime(visits['start_ts'], format="%Y-%m-%d %H:%M:%S")
visits['end_ts'] =  pd.to_datetime(visits['end_ts'], format="%Y-%m-%d %H:%M:%S")
visits.info(memory_usage='deep')

memory usage was reduced from 80 MB to 11 MB for visits table

In [ ]:
# downloading and getting to know second table
orders = pd.read_csv('/datasets/orders_log_us.csv')
orders.head()

In [ ]:
# renaming columns
orders.columns = orders.columns.str.lower().str.replace(' ', '_')
orders.head()

In [ ]:
orders.info(memory_usage='deep')

In [ ]:
orders['buy_ts'] = pd.to_datetime(orders['buy_ts'], format="%Y-%m-%d %H:%M:%S")
orders.info(memory_usage='deep')

memory usage was reduced from 4 MB to 1 MB for orders table

In [ ]:
# downloading and getting to know third table
expenses = pd.read_csv('/datasets/costs_us.csv')
expenses.head()

In [ ]:
expenses.info(memory_usage='deep')

In [ ]:
expenses['dt'] = pd.to_datetime(expenses['dt'], format='%Y-%m-%d')
expenses.info(memory_usage='deep')

memory usage was reduced from 206 KB to 60 KB for expenses table

**Checking for missing data in 3 tables**

In [ ]:
visits.stb.missing(style=True)

In [ ]:
orders.stb.missing(style=True)

In [ ]:
expenses.stb.missing(style=True)

No missing data was found

**Checking for duplicated data in 3 tables**

In [ ]:
visits.duplicated().sum()

In [ ]:
orders.duplicated().sum()

In [ ]:
expenses.duplicated().sum()

No duplicated data was found

There were no duplicated or missing data in any of the tables.  
    
Data types were optimized in each table.  
    
Column names were  changed according to naming rules  
    
The data is clean and ready for analysis.  

## . Make reports and calculate metrics:  

**1. product**

**How many people use the product (visit) every day, week, and month?**

In [ ]:
# extracting date, week and month from start_ts
visits['week']  = visits['start_ts'].astype('datetime64[W]')
visits['month']  = visits['start_ts'].astype('datetime64[M]')
visits['date'] = visits['start_ts'].astype('datetime64[D]')
visits.head()

Calculating and presenting DAU, WAU, and MAU.

In [ ]:
dau = visits.groupby(visits['date'])['uid'].nunique().reset_index().rename(columns={'uid':'n_users'})
dau.head()

In [ ]:
wau = visits.groupby(visits['week'])['uid'].nunique().reset_index().rename(columns={'uid':'n_users'})
wau.head()

In [ ]:
mau = visits.groupby(visits['month'])['uid'].nunique().reset_index().rename(columns={'uid':'n_users'})
mau.head()

Plotting  DAU, WAU, and MAU.

In [ ]:
fig = px.line(dau,x="date", y="n_users",title='DAU', labels={'n_users':'unique users'})
#adding reference line with average DAU over time
fig.add_hline(y=dau['n_users'].mean(),line_dash="dash", line_color="purple", annotation_text="average DAU",
             annotation_position="top left")
fig.show()

DAU - Graph description:  
The average daily active users is around 900.  
From June we see an increase in active users with a pick at the end of November.  
After that, the number of DAU ranges between 900-1500, and by the end of March we that the DAU is around and below average.  
(excluding a pick at the end of the data)

In [ ]:
fig = px.line(wau,x="week", y="n_users",title='WAU', labels={'n_users':'unique users'})
#adding reference line with average DAU over time
fig.add_hline(y=wau['n_users'].mean(),line_dash="dash", line_color="purple", annotation_text="average WAU",
             annotation_position="top left")
fig.show()

WAU - Graph description:  
The average weekly active users is around 5700.  
From June we see an increase in active users with a pick at the end of November.  
After that, the number of WAU ranges between average and up to 8500, and by the end of March we that the DAU is around and below average.  
The trend is similar to the DAU trend.

In [ ]:
fig = px.line(mau,x="month", y="n_users",title='MAU', labels={'n_users':'unique users'})
#adding reference line with average DAU over time
fig.add_hline(y=mau['n_users'].mean(),line_dash="dash", line_color="purple", annotation_text="average MAU",
             annotation_position="top left")
fig.show()

MAU - Graph description:  
The average monthly active users is around 23K.  
From June we see an increase in active users with a pick at November.  
After that, the number of MAU decreases, and reaches average between March and April.

In [ ]:
# Calculating mean DAU
print(f'the mean DAU is {dau.n_users.mean():.1f}')

In [ ]:
# Calculating mean WAU
print(f'the mean WAU is {wau.n_users.mean():.1f}')

In [ ]:
# Calculating mean DAU
print(f'the mean MAU is {mau.n_users.mean():.1f}')

**How many sessions are there per day?**

In [ ]:
daily_sess = visits.groupby(visits['date'])['uid'].count().reset_index().rename(columns={'uid':'n_ssesions'})
daily_sess.head()

In [ ]:
fig = px.line(daily_sess,x="date", y="n_ssesions",title='Daily Sessions', labels={'n_ssesions':'unique ssesions'})
#adding reference line with average DAU over time
fig.add_hline(y=daily_sess['n_ssesions'].mean(),line_dash="dash", line_color="purple", annotation_text="average ssesions",
             annotation_position="top left")
fig.show()

In [ ]:
daily_sess = daily_sess.merge(dau, on='date')
daily_sess.head()

In [ ]:
daily_sess['sess_per_user'] = daily_sess['n_ssesions'] / daily_sess['n_users']
daily_sess.head()

In [ ]:
fig = px.line(daily_sess,x="date", y="sess_per_user",title='Daily Sessions per User', labels={'sess_per_user':'ssesions per user'})
#adding reference line with average DAU over time
fig.add_hline(y=daily_sess['sess_per_user'].mean(),line_dash="dash", line_color="purple", annotation_text="average ssesions",
             annotation_position="top left")
fig.show()

Daily ssesions per user- Graph description:  
The mean daily ssesions per user is 1.1.  
There is no major change in the number of ssesion per user and this metric varies between 1-1.2.

**What is the length of each session?**

In [ ]:
# Calculating duration for each ssesion (in seconds)
visits['duration'] = (visits['end_ts'] - visits['start_ts']).astype('timedelta64[s]')
visits.head()

In [ ]:
# Calculating mean duration for all data
print(f'the mean duration is {visits.duration.mean():.2f}')

In [ ]:
# Calculating median duration for all data
print(f'the median duration is {visits.duration.median():.2f}')

In [ ]:
visits.duration.min()

In [ ]:
visits.sort_values(by='duration').head()

2 rows have negative values for duration.  
Probably start_ts or end_ts were logged incorrectly.

In [ ]:
# Calculating mean duration by device
visits.groupby(visits['device'])['duration'].mean()

In [ ]:
# Calculating median duration by device
visits.groupby(visits['device'])['duration'].median()

Looks like the mean and median depend on the device type.

In [ ]:
# Plotting daily sessions duration
fig=px.histogram(y=visits['duration'],x=visits['date'], title='Total daily sessions duration', labels={'y':'duration','x':'date'})
fig.show()

I will not use this graph. Instead I will make a graph with average duration.

In [ ]:
# Plotting daily sessions duration, using avg instead of sum
fig=px.histogram(y=visits['duration'],x=visits['date'], histfunc='avg', title='Average daily sessions duration', labels={'y':'duration','x':'date'})
fig.add_hline(y=visits['duration'].median(),line_dash="dash", line_color="purple", annotation_text="average duration",
             annotation_position="top left")
fig.show()

According to the first histogram, and the difference between the mean and median values I concluded that the distribution is not normal and decided to use median for the histogram Instead of mean.
I am not sure that I understand the comment.

In [ ]:
visits.duration.hist()

**What's the user retention rate?**

In [ ]:
# finding the first session for each user
first_visit = visits.groupby(['uid'])['date'].min().reset_index()
first_visit.columns = ['uid', 'first_session']
first_visit.head()

In [ ]:
# merging with the original dataset
visits = visits.merge(first_visit, how='inner',on=['uid'])
visits.head()

In [ ]:
# Creating cohorts by month
visits['cohort'] = visits['first_session'].astype('datetime64[M]')
visits.head()

In [ ]:
# Calculating age
visits['age'] = ((visits['date'] - visits['first_session']) / np.timedelta64(1,'M')).round().astype('int')
visits.head()

In [ ]:
# Active users by cohort and age
cohorts = visits.pivot_table(index='cohort',
                  columns='age',
                  values='uid',
                  aggfunc='nunique').fillna(0)
cohorts

In [ ]:
# Retention rate table
retention=cohorts.div(cohorts[0], axis=0)
retention

In [ ]:
# Plotting retention rate
fig, ax = plt.subplots(figsize=(10,10)) 
retention.index=retention.index.astype(str)
sns.heatmap(retention, annot=True, fmt='.1%', linewidths=1, linecolor='grey',  vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'Retention Rate')
plt.show()

In conclusion, the retention rate observed in our cohort analysis reveals a critical trend: a gradual decline in retention over time. With a maximum retention rate of 7.6% after one month, our analysis underscores the challenge of sustaining engagement and loyalty among our user base. 
In addition, it is noteworthy that some of the cohorts exhibited retention rates dropping below 1% within just a few months.


**2. Sales**

**When do people start buying?**

In [ ]:
# Checking nature of product (is it a one-time purchase?)
orders.uid.duplicated().sum()

There are multiple purchases per user, need to find the first order.

In [ ]:
# Finding first order for user
first_buy = orders.groupby(['uid'])['buy_ts'].min().reset_index()
first_buy.columns = ['uid', 'first_purchase']
first_buy.head()

In [ ]:
# Extracting first order date from date and time
first_buy['first_purchase_date'] = first_buy['first_purchase'].astype('datetime64[D]')
first_buy.head()

In [ ]:
# Adding first session column to data
first_buy = first_buy.merge(first_visit, how='inner',on=['uid'])
first_buy.head()

In [ ]:
# calculating conversion in days
first_buy['conversion'] = (first_buy['first_purchase_date'] - first_buy['first_session'] ).astype('timedelta64[D]').astype('int')
first_buy.head()

In [ ]:
# plotting conversion
fig = px.histogram(first_buy, x='conversion', nbins=30)
fig.show()

In [ ]:
first_buy['conversion'].describe()

In [ ]:
first_buy['conversion'].value_counts(normalize=True)

Over 68% of the users made their first purchase the same day that they first use the product.

In [ ]:
print('The overall conversion is {:.0%}'.format(orders['uid'].nunique() / visits['uid'].nunique()))

In [ ]:
# adding cohort information to the data
cohort_conversion = first_buy.merge(visits, how='left', on='uid')
cohort_conversion.head()

In [ ]:
# Checking days to conversion vy cohort
conversion_by_cohort = cohort_conversion.groupby(['cohort'])['conversion'].median().reset_index().rename(columns={'conversion':'days to conversion'})
conversion_by_cohort

The first 2 cohorts took on average over 20 days to convert.  
Most cohort on average convert on the same dau of registration.

Fast conversion signifies the efficiency and effectiveness of the sales process in persuading individuals to buy tickets. However, while fast conversion can offer benefits such as increased revenue and efficiency, it's essential to balance speed with quality, personalized service, and long-term relationship building.

**How many orders do they make during a given period of time?**

In [ ]:
# adding firat buy info to orders data
orders = orders.merge(first_buy, on='uid', how='left')

In [ ]:
# Dropping unnecessary columns
orders = orders.drop(columns=['first_purchase', 'first_session', 'conversion'])

In [ ]:
# creating column first_order_month
orders['first_order_month'] = orders['first_purchase_date'].astype('datetime64[M]')

In [ ]:
# creating column order_month
orders['order_month'] = orders['buy_ts'].astype('datetime64[M]')

In [ ]:
# Calculating cohort age using only orders
orders['age_buy'] = ((orders['order_month'] - orders['first_order_month']) / np.timedelta64(1,'M')).round().astype('int')

In [ ]:
orders.head()

In [ ]:
# creating pivot table for number of buys
orders_cohorts = orders.pivot_table(index='first_order_month',
                  columns='age_buy',
                  values='uid',
                  aggfunc='nunique').fillna(0)
orders_cohorts

In [ ]:
# creating pivot table for rate of buys
buy_rate = orders_cohorts.div(orders_cohorts[0], axis=0)
buy_rate

In [ ]:
# Plotting buy rate
fig, ax = plt.subplots(figsize=(10,10)) 
buy_rate.index=buy_rate.index.astype(str)
sns.heatmap(buy_rate, annot=True, fmt='.1%', linewidths=1, linecolor='grey',  vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'Buy Rate')
plt.show()

After the first month of purchase, the buy rate drastically drops.  
In the following months you can see minimal changes, so the buying rate ranges from 1-4%.

In [ ]:
# Getting cohorts size
cohort_sizes = orders.groupby(orders['first_order_month'])['uid'].nunique().reset_index().rename(columns={'uid':'cohort_size'})
cohort_sizes

In [ ]:
#calculating number of purchases for cohort and month
cohort=orders.groupby(['first_order_month','order_month'])['revenue'].count().reset_index()
cohort.columns=['first_order_month','month','orders']
cohort

In [ ]:
#merge cohort with cohort size
cohort = cohort.merge(cohort_sizes, on='first_order_month')
cohort

In [ ]:
# Calculating orders per buyer
cohort['orders_per_buyer'] = cohort['orders'] / cohort['cohort_size']
cohort

In [ ]:
# Calculating cohort age
cohort['age'] = ((cohort['month'] - cohort['first_order_month']) / np.timedelta64(1,'M')).round().astype('int')
cohort

In [ ]:
# cohort analysis- purchases per buyer
cohort_piv = cohort.pivot_table(index='first_order_month',
                               columns='age',
                               values='orders_per_buyer',
                               aggfunc='sum').round(2).fillna(0)
cohort_piv

In [ ]:
# Plotting purchases per buyer
fig, ax = plt.subplots(figsize=(10,10)) 
cohort_piv.index=cohort_piv.index.astype(str)
sns.heatmap(cohort_piv, annot=True, fmt='.3', linewidths=1, linecolor='grey',  vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'Purchases per Buyer')
plt.show()

The first month of buying has the highest value of purchases per buyer (>1).  
After the first month the number of purchases per buyer constantly drops, and is lesser then 1 buy per buyer.

In [ ]:
# cohort analysis- cumulative purchases per buyer
cohort_piv_cum = cohort.pivot_table(index='first_order_month',
                               columns='age',
                               values='orders_per_buyer',
                               aggfunc='sum').round(2).fillna(0).cumsum(axis=1)
cohort_piv_cum

In [ ]:
# Plotting cumulative purchases per buyer 
fig, ax = plt.subplots(figsize=(10,10)) 
cohort_piv_cum.index=cohort_piv_cum.index.astype(str)
sns.heatmap(cohort_piv_cum, annot=True, fmt='.3', linewidths=1, linecolor='grey',  vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'Cumulative Purchases per Buyer')
plt.show()

When comparing cumulative number of purchases per buyers between cohorts, we can see that the first cohort (of june 2017) is the best one in terms of purchases per buyer.

**What is the average purchase size?**

In [ ]:
# creating small table (avg revenue by first order month and order month) and calculating age
avg_cohort = orders.groupby(['first_order_month','order_month'])['revenue'].mean().reset_index()
avg_cohort['age'] = ((avg_cohort['order_month'] - avg_cohort['first_order_month']) / np.timedelta64(1,'M')).round().astype('int')
avg_cohort

In [ ]:
# creatong pivot table for avg revenue
avg_cohort_piv = avg_cohort.pivot_table(index='first_order_month',
                                        columns='age',
                                        values='revenue',
                                        aggfunc='mean').round(2).fillna('')
avg_cohort_piv

For most cohorts, the average revanue growes as the months go by.

In [ ]:
# Getting revenue statistics for each cohort
orders.groupby(['first_order_month'])['revenue'].describe()

We can see that for all cohorts the mean revenue is higher than the median.  
There are some big purchases that influance the mean- particularly in the cohorts of September and December 17.

In [ ]:
avg_rev = orders.groupby('order_month')['revenue'].mean().reset_index()
fig = px.line(avg_rev, y='revenue', x='order_month', labels={'revenue':'Avg revenue','order_month':'Month'}, title='Average Revenue per Month')
fig.add_hline(y=avg_rev['revenue'].mean(),line_dash="dash", line_color="purple", annotation_text="average revenue",
             annotation_position="top left")
fig.show()

In [ ]:
avg_rev.revenue.describe()

Average revenue is 4.8.  
The most profitable month is December and the least profitable are January and June.

**How much money do they bring? (LTV)**

In [ ]:
# creating small table (total revenue by first order month and order month)
ltv_cohort = orders.groupby(['first_order_month','order_month'])['revenue'].sum().reset_index()
ltv_cohort

In [ ]:
# merge with the cohort size and calculate age
ltv_cohort = ltv_cohort.merge(cohort_sizes, on='first_order_month')
ltv_cohort['age'] = ((ltv_cohort['order_month'] - ltv_cohort['first_order_month']) / np.timedelta64(1,'M')).round().astype('int')
ltv_cohort

In [ ]:
ltv_cohort['ltv'] = ltv_cohort['revenue'] / ltv_cohort['cohort_size']
ltv_cohort

In [ ]:
# Creatong pivot table of cumulative revenue per cohort
ltv_cohort_piv = ltv_cohort.pivot_table(index='first_order_month',
                                        columns='age',
                                        values='ltv',
                                        aggfunc='sum').cumsum(axis=1).round(2)
ltv_cohort_piv

In [ ]:
# Plotting the pivot table (LTV)
fig, ax = plt.subplots(figsize=(10,10)) 
ltv_cohort_piv.index=ltv_cohort_piv.index.astype(str)
sns.heatmap(ltv_cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='LTV')
plt.show()

In conclusion, the cohort analysis reveals a promising trend towards achieving a good lifetime value (LTV) for the business.  
After 3 month the cohort of December 17 is the most profitable cohort, but after 6 month the cohort of September 17 is the most profitable.  
The cohort of December 17 will probably be more profitable than Septembers 17 cohort after 6 month.

**3. marketing**

**How much money was spent? Overall/per source/over time**

In [ ]:
expenses

In [ ]:
# Overall marketing costs
print(f'Overall marketing costs is {expenses.costs.sum():.2f}')

In [ ]:
# Marketing costs per source
ex_by_source = expenses.groupby('source_id')['costs'].sum().reset_index()
ex_by_source

In [ ]:
# Creating month column from dt
expenses['month'] = expenses['dt'].astype('datetime64[M]')
expenses

In [ ]:
# Marketing costs per month
total_monthly = expenses.groupby('month')['costs'].sum().reset_index()
total_monthly

In [ ]:
# Marketing costs per source and month
expenses_by_month_source = expenses.groupby(['source_id','month'])['costs'].sum().reset_index()
expenses_by_month_source

In [ ]:
# Marketing costs per source and month as pivot table
marketing_pivot = expenses.pivot_table(index='month',
                     columns='source_id',
                     values='costs',
                     aggfunc='sum')
marketing_pivot

In [ ]:
# Plotting the pivot table
fig, ax = plt.subplots(figsize=(10,10)) 
marketing_pivot.index=marketing_pivot.index.astype(str)
sns.heatmap(marketing_pivot, annot=True, fmt='.0f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='Marketing costs')
plt.show()

In [ ]:
# Calculating total expenses per day
total_daily = expenses.groupby('dt')['costs'].sum().reset_index()
total_daily

In [ ]:
# Plotting total expenses over time
fig = px.line(total_daily, x="dt", y="costs", title='Total Expenses Over Time- Daily', labels={'costs':'Cost','dt':'Date'})
fig.show()

In [ ]:
# Plotting total expenses over time
fig = px.line(total_monthly, x="month", y="costs", title='Total Expenses Over Time- Monthly', labels={'costs':'Cost'})
fig.show()

In [ ]:
# Plotting total expenses over time by source
fig = px.line(expenses, x="dt", y="costs", color='source_id')
fig.show()

In [ ]:
# Plotting total expenses by month and source
fig = px.line(expenses_by_month_source, x='month', y="costs", color='source_id', title='Total Expenses Over Time- Monthly')
fig.show()

August exhibits the lowest marketing expenses compared to other months, with costs gradually rising until December, after which there's a noticeable decline in marketing expenses until May.  
Expenses are particularly high by the end of the year (Nov'-Dec'), suggesting an expensive marketing event. 
  
It's evident that Source 3 incurs the highest marketing expenses among all sources, with its costs significantly surpassing those of the others, while the least costly sources are 9 and 10.  
However, despite these differences, all sources generally follow the same trend over time.

When comparing the DAU plot to the 'total expenses over time- daily' plot, and the MAU to the 'total expenses over time- monthly' plot it is evident that the correlate almost perfectly, suggesting that marketing expensees are immidiatly translated to active users.  
There is no visible correlation between marketing expenses and revenue.

**How much did customer acquisition from each of the sources cost?**

**CAC by source**

In [ ]:
# Finding first source for each user (from visits table)
first_source = visits.sort_values('date').groupby('uid').first()['source_id'].reset_index()
first_source.columns=['uid','first_source']
first_source.head()

In [ ]:
# Adding first_source column to orders table
orders = orders.merge(first_source, on='uid')
orders.head()

In [ ]:
# Calculating number of users from each source
orders_by_source = orders.groupby('first_source')['uid'].nunique().reset_index().rename(columns={'first_source':'source_id', 'uid':'users'})
orders_by_source
# there are no buyers from source 6, and 8

In [ ]:
ex_by_source
# there are no marketing expenses for sources 6-8

In [ ]:
# Merging users and costs date per source
orders_by_source = orders_by_source.merge(ex_by_source, on='source_id', how='outer')
orders_by_source

In [ ]:
# Calculating CAC per source
orders_by_source['CAC'] = orders_by_source['costs'] / orders_by_source['users']
orders_by_source

In [ ]:
orders_by_source = orders_by_source.dropna()
orders_by_source

In [ ]:
# Converting source _id into str for visualization
orders_by_source['source_id'] = orders_by_source['source_id'].astype(str)
orders_by_source.info()

In [ ]:
# Plotting CAC per source
fig = px.bar(orders_by_source, x="source_id", y="CAC", title='CAC by Source')
fig.add_hline(y=orders_by_source['CAC'].mean(),line_dash="dash", line_color="purple", annotation_text="average CAC",
             annotation_position="top left")
fig.show()

In [ ]:
orders_by_source.CAC.describe()

The mean and median for CAC by source are 7.9, and 6.2 respectively.  
The highest CAC value is almost 14 and belong to source 3.  
The lowest CAC value is 4.5 and belong to source 10.  
There ware no marketing expenses for sources 6-8.

**CAC by month**

In [ ]:
# Creating table from orders (cohort_size) and expenses
total_monthly = total_monthly.merge(cohort_sizes, left_on='month', right_on='first_order_month')
total_monthly

In [ ]:
# Calculating CAC by month
total_monthly['CAC'] = total_monthly['costs'] / total_monthly['cohort_size']
total_monthly

In [ ]:
# Plotting CAC by month
fig = px.line(total_monthly, x="month", y="CAC", title='CAC')
fig.add_hline(y=total_monthly['CAC'].mean(),line_dash="dash", line_color="purple", annotation_text="average CAC",
             annotation_position="top left")
fig.show()

In [ ]:
total_monthly['CAC'].describe()

The avg CAC is around 9.  
The highest cac value was on Aug 17, and the lowest was on May 18.

In [ ]:
# Marketing expenses by source and month
expenses_by_month_source

In [ ]:
# Orders revenue by source and month
orders_by_month_source = orders.groupby(['first_source','first_order_month'])['uid'].nunique().reset_index()
orders_by_month_source.columns=['source_id','month','buyers']
orders_by_month_source

In [ ]:
# Merging both tables
CAC_per_month_source = expenses_by_month_source.merge(orders_by_month_source, how='left', on=['month','source_id'])
CAC_per_month_source

In [ ]:
# Calculating CAC by source and month
CAC_per_month_source['CAC'] = CAC_per_month_source['costs'] / CAC_per_month_source['buyers']
CAC_per_month_source

In [ ]:
# Plotting CAC over time by source
fig = px.line(CAC_per_month_source, x="month", y="CAC",color='source_id',title='CAC', labels={'source_id':'Source','month':'Month'})
fig.show()

Customer Acquisition Cost (CAC) varies across different sources and months, providing insights into our marketing performance.

When examining CAC by source, we observe that the mean and median values stand at 7.9 and 6.2, respectively. Notably, Source 3 demonstrates the highest CAC, nearly reaching 14, while Source 10 boasts the lowest, at 4.5. Interestingly, Sources 6 through 8 incurred no marketing expenses during this period.

Shifting focus to CAC by month, we find that the average CAC hovers around 9. August 2017 marked the peak CAC, contrasting with the lowest recorded in May 2018.

Examining CAC by both month and source reveals that Sources 2 and 3 consistently exhibit substantially higher CAC values compared to other sources across multiple months.

**How worthwhile were the investments? (ROI)**

**ROI by cohort**

In [ ]:
# Extracting needed column from total_monthly
monthly_CAC_ROI = total_monthly[['first_order_month', 'CAC']]
monthly_CAC_ROI

In [ ]:
# Merging tables to calculate ROI
ROI = ltv_cohort.merge(monthly_CAC_ROI,on=['first_order_month'],how='left')
ROI

In [ ]:
# Calculating ROI
ROI['ROI'] = ROI['ltv'] / ROI['CAC']
ROI

In [ ]:
# Creating ROI pivot table
roi_piv = ROI.pivot_table(index='first_order_month', 
                          columns='age', 
                          values='ROI', 
                          aggfunc='mean').cumsum(axis=1).round(2) 
roi_piv

In [ ]:
# Creating ROI heat map
fig, ax = plt.subplots(figsize=(10,10)) 
roi_piv.index=roi_piv.index.astype(str)
sns.heatmap(roi_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='ROI per cohort')
plt.show()

In [ ]:
roi_piv.describe().round(2)

The cohort of Sep' 17 is the best cohort in terms of ROI- rose above 100% after 3 months.  
In second place we have the cohort of June 17.  
In average ROI got to 100% after 8 month.

**ROI by source**

In [ ]:
# Calculating number of buyers and sum of revenue per source
ltv_per_source = orders.groupby(['first_source'])[['uid','revenue']].agg({'uid':'nunique','revenue':'sum'}).reset_index()
ltv_per_source.columns = ['source','buyers','revenue']
ltv_per_source

In [ ]:
# Calculating LTV per source
ltv_per_source['ltv'] = ltv_per_source['revenue']/ltv_per_source['buyers']
ltv_per_source

In [ ]:
# Calculating marketing expenses per source
marketing_per_source = expenses.groupby(['source_id'])['costs'].sum().reset_index().rename(columns={'source_id':'source'})
marketing_per_source

In [ ]:
# merging tables for calculations
roi_per_source = marketing_per_source.merge(ltv_per_source, on='source')
roi_per_source

In [ ]:
# Calculating CAC and ROI per source
roi_per_source['CAC'] = roi_per_source['costs'] / roi_per_source['buyers']
roi_per_source['roi'] = roi_per_source['ltv'] / roi_per_source['CAC']
roi_per_source

In [ ]:
# Plotting CAC and ROI per source
fig = px.bar(roi_per_source, x='source', y='roi', labels={'source':'Source','roi':'ROI'}, title= 'ROI per Source')
fig.update_xaxes(type='category')
fig.show()

When scrutinizing the Return on Investment (ROI) per source, it becomes apparent that only a select few sources yield returns that cover the marketing expenses incurred. Specifically, sources 1, 2, 5, and 9 demonstrate ROI significant enough to recoup their corresponding marketing investments.  
  
In contrast, sources 3, 4, and 10 fail to generate returns that match or exceed the initial marketing expenditures. Despite the resources allocated to these sources, they do not yield sufficient returns to justify the investment made in marketing efforts.

**Final Conclusions:**  
In this project, I conducted an analysis on Yandex.Afisha data encompassing customer visits, orders, and marketing expenses. The insights drawn from this analysis shed light on various aspects of user behavior, purchasing patterns, and marketing performance, ultimately informing key business decisions.

**User Engagement and Retention:**
The analysis revealed essential metrics regarding user engagement and retention. We found that the average daily active users reached 908, with weekly and monthly metrics standing at 5724 and 23,228, respectively. Additionally, the average daily sessions per user were 1.1, with a notable difference in session duration between desktop and touch devices. However, the most critical observation was the gradual decline in retention rates over time, with some cohorts experiencing rates dropping below 1% within a few months. This highlights the challenge of sustaining engagement and loyalty among users and underscores the need for targeted retention strategies.

**Conversion, Purchases, and Lifetime Value:**
Examining conversion rates and purchasing behavior revealed valuable insights into user acquisition and retention. While the overall conversion rate stood at 16%, it was observed that the first two cohorts took longer to convert, with over 68% of users making their first purchase on the same day they first used the product. However, the buying rate drastically dropped after the first month, with minimal changes in subsequent months. Despite this, there was a promising trend towards achieving a good lifetime value (LTV) for the business, with cohorts showing growth in purchase size over time.

**Marketing Expenses, CAC, and ROI:**
Analyzing marketing expenses, customer acquisition costs (CAC), and return on investment (ROI) provided critical insights into the effectiveness of marketing strategies. It was evident that marketing expenses varied across different sources and months, with Source 3 incurring the highest expenses. Despite this, all sources generally followed the same trend over time. Furthermore, while sources 1, 2, 5, and 9 demonstrated significant ROI, sources 3, 4, and 10 failed to generate sufficient returns to justify the marketing investments made.

**Business Conclusions:**
Overall, the analysis highlights the importance of targeted retention strategies to combat declining retention rates and sustain user engagement and loyalty over time. Additionally, optimizing marketing strategies to focus on sources with higher ROI can maximize returns and improve overall profitability. By leveraging these insights, businesses can make informed decisions to drive growth and success in the competitive marketplace.